## Trackmate Trajectory Analysis

(1) Velocities Distribuition Directly from Spot Displacement <br>
(2) Velocitiy distribution from fitting MSD curves individually <br>
(3) Average velocity from fitting a mean MSD curve <br>
(4) Directionality from velocity auto-correlation analysis

### Core Function
** Run the cell below to call all the necessary functions <br> **
** At the bottom, one can run the analysis for a single xml file or in batch **

In [14]:
from analyze_tracks import read, msd, velocity, utils
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

#packages for batch processing
import os
import glob
import sys

# packages for a progress bar
from ipywidgets import FloatProgress
from IPython.display import display

# inline display
#get_ipython().magic('matplotlib inline')

# Needed for autoreload when ext python module has changed
get_ipython().magic('matplotlib qt5')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

def analyze_tracks(filename, clip = 0.5, plot_every = 20):
    '''runs all type of analyses and saves all plots into a
    single pdf file and all data as sheets of one excel book'''
    
    plt.close("all")
    
    table_tracks, frame_interval, time_units, space_units = read.tm_xml_tracks(filename)

    print('\nPhysical units: {}, {}'.format(space_units, time_units))
    print('Number of tracks: {}'.format(len(table_tracks.TRACK_ID.unique())))
    print('Frame interval: {} \n'.format(frame_interval))

    # savename = filename.split(sep='/')[-1][:-4]
    with PdfPages(filename[:-4] + '_All_figs.pdf') as pdf:
        
        #table_tracks = table_tracks[:1000] # temporary hack to analyze only a few trajectories
        
        velo_dist = velocity.velocities_distribution(table_tracks, frame_interval);
        pdf.savefig(bbox_inches="tight")
            
        all_msds_vel, all_msd_curves = msd.single_track_analysis(table_tracks, frame_interval, plot_every);
        pdf.savefig(bbox_inches="tight")

        V, D, msd_fit = msd.msd_velocity_analysis(table_tracks, frame_interval, clip = 0.5)
        pdf.savefig(bbox_inches="tight")
        
        vcorr_data = velocity.compute_directionality(table_tracks, frame_interval)
        pdf.savefig(bbox_inches="tight")
                   
        with pd.ExcelWriter(filename[:-4] + '_Trackmate_Trajectory_Analysis.xlsx') as excel_sheet:
            
            pd.Series(velo_dist).to_excel(excel_sheet, sheet_name = 'vels_dist', index=False, header = False)
            pd.Series(all_msds_vel).to_excel(excel_sheet, sheet_name = 'msd_vels_hist', index=False, header = False)
            all_msd_curves.to_excel(excel_sheet, sheet_name = 'all_msd_curves', index=False, header = False)
            msd_fit.to_excel(excel_sheet, sheet_name = 'averaged_MSD ', index=False)
            vcorr_data.to_excel(excel_sheet, sheet_name = 'vcorr', index=False)
    
    print('Done!')

# small command to prevent print statments during batch processing

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout

# running in batch
def analyze_tracks_batch(files_dir, clip = 0.5):
    
    data_path = os.path.join(files_dir,'*Tracks.xml')
    files = glob.glob(data_path)
    
    if len(files) == 0:
        print('File directory is empty!')
    
    else:
        
        bar = FloatProgress(description = 'Progress ...', bar_style= 'info', max = len(files) )
        display(bar)

        for i,xml_file in enumerate(files):

            print('Processing '+ xml_file.split('\\')[-1])
            
            try:
                
                with HiddenPrints(): analyze_tracks(xml_file) #this blocks print statments while running the function
                bar.value += 1
            
            except:
                print ('\t !!! This file cannot be processed !!!')
                continue

    print('\nAll Done! Yay!')

### Analyze Trajectories
** Select a TrackMate XML File <br> **
** Plots the output of each analysis and saves figs (pdf) & data (excel) **

In [15]:
filename = 'C:/Users/pcaldas/Desktop/Data Analysis/Methods Paper/alberts data/C2-Grow_Tracks.xml'
analyze_tracks(filename, clip = 0.5, plot_every = 20)


Physical units: micron, sec.
Number of tracks: 229
Frame interval: 1.0 

Processing Velocities Distribution ...
Processing Single Track MSD Analysis ...


C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Processing Averaged MSD Analysis ...
Processing Velocity Autocorrelation ...
Done!


### Analyze Trajectories In Batch
** Select a folder containing TrackMate XML Files <br> **
** Save all figs (pdf) all data (excel) **

In [ ]:
files_dir = "ipynb/files"
analyze_tracks_batch(files_dir, clip = 0.5)